In [1]:
#!/usr/bin/python

import sys

def writelp(lpfilename, prices, numsec, numscen):

    try:
        lpfile = open(lpfilename, 'w') # opens the file
    except IOError:
        print("Cannot open LP file %s for writing\n" % lpfilename)
        return 1

    print "now writing LP to file", lpfilename
    lpfile.write("Minimize ")
    j = 0
    while j <= numsec:
        if prices[j] >= 0:
            lpfile.write("+ ")
        lpfile.write(str(prices[j]) + " x" + str(j)+" ")
        # str converts argument into string;  " +" concatenates strings
        j += 1
    lpfile.write("\nSubject to\n")

    k = 0
    while k <= numscen:
        # write constraint for scenario k
        lpfile.write("scen_" + str(k) +": ")
        j = 0
        while j <= numsec:
            index = k*(1 + numsec) + j
            if prices[index] >= 0:
                lpfile.write("+ ")
                lpfile.write(str(prices[index]) + " x" + str(j)+" ")
            j += 1
        lpfile.write(" >= 0\n")
        k += 1

    lpfile.write("Bounds\n")
    j = 0
    while j <= numsec:
        lpfile.write("-1 <= x" + str(j) + " <= 1\n")
        j += 1
    lpfile.write("End\n")

    print "closing lp file"
    lpfile.close()
      
    return 0

In [2]:
#!/usr/bin/python

#now open and read data file
try:
    datafile = open('arb.dat', 'r') # opens the data file
except IOError:
    sys.exit("Cannot open file %s\n" % sys.argv[1])


lines = datafile.readlines();
datafile.close()

#print lines[0]
firstline = lines[0].split()
#print "first line is", firstline

numsec = int(firstline[1])
numscen = int(firstline[3])
r = float(firstline[5])
print "\n"
print "number of securities:", numsec,"number of scenarios", numscen,"r",r
print "\n"

#allocate prices as one-dim array
total = (1 + numsec)*(1 + numscen)
print "total allocation:",  total
p = [0]*total
k = 0
# line k+1 has scenario k (0 = today)
while k <= numscen:
    thisline = lines[k + 1].split()
    # should check that the line contains numsec + 1 words

    p[k*(1 + numsec)] = 1 + r*(k != 0) # handles the price of cash
    j = 1
    while j <= numsec:
        value = float(thisline[j])
        p[k*(1 + numsec) + j] = value
        # print ">>", "k",k, "j",j, k*(1 + numsec) + j
        j += 1
    k += 1

#now write LP file, now done in a separate function (should read data this way, as well)

lpwritecode = writelp('test.lp', p, numsec, numscen)

#now solve lp 

#lpsolvecode = lpsolver(sys.argv[2], "test.log")

#print "solved LP at", sys.argv[2],"with code", lpsolvecode



number of securities: 6 number of scenarios 3 r 0.05


total allocation: 28
now writing LP to file test.lp
closing lp file


In [19]:
def writelp(lpfilename, prices, deviations, numsec, numscen):
    try:
        lpfile = open(lpfilename, 'w') # opens the file
    except IOError:
        print("Cannot open LP file %s for writing\n" % lpfilename)
        return 1

    print "now writing LP to file", lpfilename
    lpfile.write("Minimize ")

    j = 0
    #Objective
    while j <= numsec:
        if prices[0][j] >= 0:
            lpfile.write("+ ")
            lpfile.write(str(prices[0][j]) + " x" + str(j)+" ")
        j += 1
    lpfile.write("\nSubject to\n")

    k = 1
    #dual constraints
    while k <= numscen:
        lpfile.write("A" + str(k) +": ")
        j = 0
        while j <= numsec:
            if prices[k][j] >= 0:
                lpfile.write("+ ")
                lpfile.write(str(prices[k][j]) + " u" + str(k) + str(j) + " -")
                lpfile.write(str(deviations[k][j]) + " u" + str(k) + str(j) + " -")
                lpfile.write(str(prices[k][j]) + " v" + str(k) + str(j) + " -")
                lpfile.write(str(deviations[k][j]) + " v" + str(k) + str(j) + " ")
            j += 1
        lpfile.write(">= 0\n")
        k += 1

    k = 1
    while k <= numscen:
        j = 0
        while j <= numsec:
            lpfile.write("B" + str(k) + str(j) +": ")
            lpfile.write("u" + str(k) + str(j) + " -")
            lpfile.write(" v" + str(k) + str(j) + " -")
            lpfile.write(" x" + str(j) + " ")
            lpfile.write("= 0\n")
            j += 1
        k += 1

    lpfile.write("Bounds\n")
    j = 0
    while j <= numsec:
        lpfile.write("-1 <= x" + str(j) + " <= 1\n")
        k = 1
        while k <= numscen:
            lpfile.write("u" + str(k) + str(j) + " >= 0\n")
            lpfile.write("v" + str(k) + str(j) + " >= 0\n")
            k += 1
        j += 1
    lpfile.write("End\n")

    print "closing lp file"
    lpfile.close()

    return 0

In [20]:
import numpy
prices = [[1,1,1],[1,1,1],[1,1,1]]
deviations = [[0.5,0.5,0.5],[0.5,0.5,0.5],[0.5,0.5,0.5]]

In [21]:
test = writelp('test1.lp', prices, deviations, 2, 2)

now writing LP to file test1.lp
closing lp file
